In [2]:
import kivy
from kivy.app import App 
from kivy.uix.label import Label
from kivy.uix.gridlayout import GridLayout
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.widget import Widget
from kivy.properties import ObjectProperty


class MyGrid(Widget):
    name = ObjectProperty(None)
    email = ObjectProperty(None)

    def btn(self):  
        print("Name:", self.name.text, "email:", self.email.text)
        self.name.text = ""
        self.email.text = ""   
        




#Call in App Class (Looks for class name in kivy file and runs it in python)
class MyApp (App):
    def build (self):
        return MyGrid()

#Run Program 

if __name__ == "__main__":
    MyApp().run()





Name: ddd email: 
